# Imports
import sys
import os
import importlib
sys.path.append(os.path.abspath('.'))

# Import and reload the module to ensure latest version
import simulation_utils
importlib.reload(simulation_utils)

from simulation_utils import (
    load_and_filter_data,
    run_single_fund_simulation,
    plot_simulation_results,
    print_simulation_summary
)
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Imports completed successfully!")

In [37]:
# Imports
import sys
import os
sys.path.append(os.path.abspath('.'))

from simulation_utils import (
    load_and_filter_data,
    run_single_fund_simulation,
    plot_simulation_results,
    print_simulation_summary
)
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Imports completed successfully!")

Imports completed successfully!


In [38]:
# Simulation Parameters
SIMULATION_PARAMS = {
    'starting_capital': 10000,           # Initial capital per fund
    'start_date': '2025-01-01',          # Simulation start date
    'max_duration_days': 365,            # Maximum simulation duration
    'days_before': 1,                    # Days before resolution to invest
    'min_prob_7d': 0.90,                 # Minimum probability at 7 days
    'min_prob_current': 0.90,            # Minimum probability at investment day
    'ending_factor_start': 0.05,         # Starting ending factor (5% chance)
    'ending_factor_increment': 0.001,    # Daily increment (0.1%)
    'skew_factor': 0.4,                  # Left skew factor (higher = more bias toward closer markets)
    'num_simulations': 100                # Number of simulation runs (reduced for speed testing)
}

print("Simulation Parameters:")
print("=" * 40)
for param, value in SIMULATION_PARAMS.items():
    print(f"{param}: {value}")

print(f"\nEnding factor will reach {SIMULATION_PARAMS['ending_factor_start'] + SIMULATION_PARAMS['ending_factor_increment'] * SIMULATION_PARAMS['max_duration_days']:.3f} after {SIMULATION_PARAMS['max_duration_days']} days")
print(f"Skew factor {SIMULATION_PARAMS['skew_factor']} controls market selection bias (higher = prefer closer markets)")
print(f"Note: Starting with {SIMULATION_PARAMS['num_simulations']} sims for speed testing. Increase num_simulations for final analysis.")

Simulation Parameters:
starting_capital: 10000
start_date: 2025-01-01
max_duration_days: 365
days_before: 1
min_prob_7d: 0.9
min_prob_current: 0.9
ending_factor_start: 0.05
ending_factor_increment: 0.001
skew_factor: 0.4
num_simulations: 100

Ending factor will reach 0.415 after 365 days
Skew factor 0.4 controls market selection bias (higher = prefer closer markets)
Note: Starting with 100 sims for speed testing. Increase num_simulations for final analysis.


In [39]:
# Run Simulations
import time

print(f"Running {SIMULATION_PARAMS['num_simulations']:,} single fund simulations...")
print("Timing performance for optimization...\n")

start_time = time.time()
results = []

# Run simulations with progress bar
for i in tqdm(range(SIMULATION_PARAMS['num_simulations']), desc="Running simulations"):
    # Use different random seed for each simulation
    result = run_single_fund_simulation(
        df=df,
        starting_capital=SIMULATION_PARAMS['starting_capital'],
        start_date=SIMULATION_PARAMS['start_date'],
        max_duration_days=SIMULATION_PARAMS['max_duration_days'],
        days_before=SIMULATION_PARAMS['days_before'],
        min_prob_7d=SIMULATION_PARAMS['min_prob_7d'],
        min_prob_current=SIMULATION_PARAMS['min_prob_current'],
        ending_factor_start=SIMULATION_PARAMS['ending_factor_start'],
        ending_factor_increment=SIMULATION_PARAMS['ending_factor_increment'],
        skew_factor=SIMULATION_PARAMS['skew_factor'],
        target_return=None,  # No target return for baseline simulation
        random_seed=i  # Different seed for each simulation
    )
    results.append(result)

end_time = time.time()
elapsed = end_time - start_time

print(f"\nCompleted {len(results):,} simulations in {elapsed:.2f} seconds!")
print(f"Average time per simulation: {elapsed/len(results):.4f} seconds")
print(f"Projected time for 1000 simulations: {elapsed*10:.1f} seconds ({elapsed*10/60:.1f} minutes)")

if elapsed > 30:  # If still slow, provide optimization suggestions
    print(f"\n⚠️  Still running slow. Consider:")
    print(f"   • Reducing max_duration_days from {SIMULATION_PARAMS['max_duration_days']} days")
    print(f"   • Increasing ending_factor_increment from {SIMULATION_PARAMS['ending_factor_increment']}")
    print(f"   • Using fewer probability columns in data filtering")

Running 100 single fund simulations...
Timing performance for optimization...



Running simulations:   0%|          | 0/100 [00:00<?, ?it/s]


TypeError: run_single_fund_simulation() got an unexpected keyword argument 'skew_factor'

In [28]:
# Print Summary Statistics
print_simulation_summary(results)

SIMULATION SUMMARY
Total Simulations: 1,000
Went Bust: 44 (4.4%)
Positive Returns: 956 (95.6%)

FINAL CAPITAL STATISTICS:
Mean: $10,715.94
Median: $10,959.06
Std Dev: $2,455.86
Min: $0.00
Max: $16,392.45

RETURN STATISTICS:
Mean: 7.2%
Median: 9.6%
Std Dev: 24.6%
Min: -100.0%
Max: 63.9%

TRADE STATISTICS:
Mean Trades: 8.7
Median Trades: 8.0
Min Trades: 1
Max Trades: 23

RETURN PERCENTILES:
5th percentile: 0.4%
10th percentile: 1.9%
25th percentile: 4.8%
75th percentile: 16.3%
90th percentile: 23.4%
95th percentile: 29.3%


In [ ]:
# Plot Results
plot_simulation_results(results, title="Single Fund Simulation Results")

In [ ]:
# Additional Analysis: Sample Capital Evolution
print("Sample Capital Evolution Curves")
print("=" * 40)

# Plot capital evolution for first 10 simulations
fig, ax = plt.subplots(figsize=(14, 8))

sample_size = min(10, len(results))
for i in range(sample_size):
    if results[i]['daily_capital']:
        daily_data = pd.DataFrame(results[i]['daily_capital'])
        ax.plot(daily_data['day'], daily_data['capital'], 
               alpha=0.7, linewidth=1.5, label=f'Sim {i+1}')

ax.axhline(y=SIMULATION_PARAMS['starting_capital'], color='red', 
          linestyle='--', linewidth=2, label='Starting Capital')
ax.set_xlabel('Simulation Day')
ax.set_ylabel('Capital ($)')
ax.set_title('Sample Capital Evolution Over Time')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')  # Log scale to better see both growth and decline

plt.tight_layout()
plt.show()

In [ ]:
# Detailed Trade Analysis
print("Trade Analysis")
print("=" * 40)

# Collect all trades from all simulations
all_trades = []
for i, result in enumerate(results):
    for trade in result['trades']:
        trade_copy = trade.copy()
        trade_copy['simulation_id'] = i
        all_trades.append(trade_copy)

trades_df = pd.DataFrame(all_trades)

if len(trades_df) > 0:
    print(f"Total trades across all simulations: {len(trades_df):,}")
    print(f"Average trades per simulation: {len(trades_df) / len(results):.1f}")
    
    # Win rate analysis
    win_rate = trades_df['outcome'].mean()
    print(f"\nOverall win rate: {win_rate:.1%}")
    
    # Average probability of trades
    avg_prob = trades_df['probability'].mean()
    print(f"Average market probability: {avg_prob:.1%}")
    
    # Return distribution per trade
    avg_return_per_trade = trades_df['return'].mean()
    print(f"Average return per trade: {avg_return_per_trade:.1%}")
    
    # Plot probability distribution of selected markets
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    ax1.hist(trades_df['probability'], bins=50, alpha=0.7, edgecolor='black')
    ax1.axvline(avg_prob, color='red', linestyle='--', 
               label=f'Mean: {avg_prob:.1%}')
    ax1.set_xlabel('Market Probability')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Market Probabilities in Trades')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.hist(trades_df['return'], bins=50, alpha=0.7, edgecolor='black')
    ax2.axvline(avg_return_per_trade, color='red', linestyle='--',
               label=f'Mean: {avg_return_per_trade:.1%}')
    ax2.axvline(0, color='black', linestyle='-', alpha=0.5, label='Break-even')
    ax2.set_xlabel('Return per Trade')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Returns per Trade')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades found in simulations.")

In [ ]:
# Export Results (Optional)
EXPORT_RESULTS = False  # Set to True to export results

if EXPORT_RESULTS:
    # Create summary DataFrame
    summary_data = []
    for i, result in enumerate(results):
        summary_data.append({
            'simulation_id': i,
            'final_capital': result['final_capital'],
            'total_return': result['total_return'],
            'num_trades': result['num_trades'],
            'went_bust': result['went_bust'],
            'ending_reason': result['ending_reason'],
            'simulation_days': result['simulation_days']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Export to CSV
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    summary_filename = f'single_fund_results_{timestamp}.csv'
    summary_df.to_csv(summary_filename, index=False)
    
    print(f"Results exported to {summary_filename}")
    print(f"Summary statistics saved for {len(summary_df):,} simulations")
else:
    print("Results export disabled. Set EXPORT_RESULTS = True to export.")